<a href="https://colab.research.google.com/github/gamountainmama/data_dames/blob/main/ed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("https://gamountainmama.github.io/data_dames/8th%20Grade%20Math/Math%208%202018.csv")
df.head()

,Gender,Ethnicity,Absences,SPED,504,ED,SST,Gifted,Scale Score,Achievement Level,"Numbers, Expressions, and Equations",Algebra and Functions,Geometry,Statistics and Probability,Norm-Referenced Score
0,M,W,13.0,Y,N,N,N,N,421,0,1,1,1,1,12
1,M,W,32.0,Y,N,N,N,N,459,0,1,1,1,1,5
2,F,W,4.0,N,Y,Y,Y,N,464,0,1,1,1,1,59
3,F,W,30.0,Y,N,Y,N,N,464,0,1,1,1,1,52
4,F,W,2.0,Y,N,N,N,N,464,0,1,1,1,1,52


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(columns='Norm-Referenced Score', axis=1)
df = df.drop(columns='Scale Score', axis=1)
df = df.drop(columns='Numbers, Expressions, and Equations', axis=1)
df = df.drop(columns='Algebra and Functions', axis=1)
df = df.drop(columns='Geometry', axis=1)
df = df.drop(columns='Statistics and Probability', axis=1)
df.head()

,Gender,Ethnicity,Absences,SPED,504,ED,SST,Gifted,Achievement Level
0,M,W,13.0,Y,N,N,N,N,0
1,M,W,32.0,Y,N,N,N,N,0
2,F,W,4.0,N,Y,Y,Y,N,0
3,F,W,30.0,Y,N,Y,N,N,0
4,F,W,2.0,Y,N,N,N,N,0


In [3]:
# Determine the number of unique values in each column.
df.dtypes

Gender                object
Ethnicity             object
Absences             float64
SPED                  object
504                   object
ED                    object
SST                   object
Gifted                object
Achievement Level      int64
dtype: object

In [4]:
# Convert categorical data to numeric with `pd.get_dummies`
num_data = pd.get_dummies(df)
num_data.head()

,Absences,Achievement Level,Gender_F,Gender_M,Ethnicity_H,Ethnicity_M,Ethnicity_W,SPED_N,SPED_Y,504_N,504_Y,ED_N,ED_Y,SST_N,SST_Y,Gifted_N
0,13.0,0,0,1,0,0,1,0,1,1,0,1,0,1,0,1
1,32.0,0,0,1,0,0,1,0,1,1,0,1,0,1,0,1
2,4.0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,1
3,30.0,0,1,0,0,0,1,0,1,1,0,0,1,1,0,1
4,2.0,0,1,0,0,0,1,0,1,1,0,1,0,1,0,1


In [5]:
# Split our preprocessed data into our features and target arrays
X = num_data.drop('Achievement Level', axis=1).values
y = num_data['Achievement Level'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input = X_train_scaled.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=input))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                160       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 171
Trainable params: 171
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# Train the model
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.hdf5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

fit_model = nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, callbacks=[checkpoint])

Epoch 1/100
2/2 [==============================] - 4s 132ms/step - loss: 0.6584 - accuracy: 0.6167
Epoch 2/100
2/2 [==============================] - 0s 57ms/step - loss: 0.6517 - accuracy: 0.6167
Epoch 3/100

Epoch 3: accuracy improved from -inf to 0.56250, saving model to best_model.hdf5
2/2 [==============================] - 0s 22ms/step - loss: 0.6454 - accuracy: 0.6333
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 0.6395 - accuracy: 0.6333
Epoch 5/100
1/2 [==============>...............] - ETA: 0s - loss: 0.5731 - accuracy: 0.5625
Epoch 5: accuracy improved from 0.56250 to 0.63333, saving model to best_model.hdf5
2/2 [==============================] - 0s 219ms/step - loss: 0.6336 - accuracy: 0.6333
Epoch 6/100
2/2 [==============================] - 0s 29ms/step - loss: 0.6279 - accuracy: 0.6500
Epoch 7/100
2/2 [==============================] - 0s 66ms/step - loss: 0.6216 - accuracy: 0.6500
Epoch 8/100

Epoch 8: accuracy improved from 0.63333 to 0.68750, 

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 [==============================] - 1s 1s/step - loss: nan - accuracy: 0.9048
Loss: nan, Accuracy: 0.9047619104385376
